# NNodely Documentation - Training

Here are listed all the modalities that can be used to perform the training process of a neural network model.

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 1.0 MB/s eta 0:00:0000:0100:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 1.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of nnodely to determine which version is compatible with other requirements. This could take a while.
  Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached conto

In [4]:
## Create a neural network model and Load a dataset
in1 = Input('in1')
target = Input('target')
relation = Fir(in1.tw(0.05))
output = Output('out', relation)

model = Modely(visualizer=TextVisualizer())
model.addMinimize('error', output, target.last())
model.addModel('model', output)
model.neuralizeModel(0.01)

train_folder = 'data'
data_struct = ['in1', '', 'target']
model.loadData(name='dataset', source=train_folder, format=data_struct, skiplines=1)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.01, 'ns': [5, 0], 'ntot': 5},
 'Inputs': {'in1': {'dim': 1,
                    'ns': [5, 0],
                    'ntot': 5,
                    'sw': [0, 0],
                    'tw': [-0.05, 0]},
            'target': {'dim': 1,
                       'ns': [1, 0],
                       'ntot': 1,
                       'sw': [-1, 0],
                       'tw': [0, 0]}},
 'Minimizers': {'error': {'A': 'out', 'B': 'SamplePart7', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Fir4'},
 'Parameters': {'PFir2p': {'dim': 1, 'tw': 0.05}},
 'Relations': {'Fir4': ['Fir', ['TimePart3'], 'PFir2p', None, 0],
               'SamplePart7': ['SamplePart', ['target'], [-1, 0]],
               'TimePart3': ['TimePart', ['in1'], [-0.05, 0]]},
 'States': {}}
============================ nnodely Model Dataset =============================
Dataset Na

## Basic Parameters and Usage

With the `trainModel` function the user can perform a training process by setting different parameters of training such as:

- models to train
- training and validation dataset
- number of epochs of training
- data shuffling
- train and validation batch size
- learning rate

And many more advanced settings.. 

If no parameters are specified, the function will train all the built models using all the loaded dataset with the default parameters.

In [5]:
training_parameters = model.trainModel()

======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                100
update per epochs:            1.0
└>(n_samples-batch_size)/batch_size+1
shuffle data:                 True
train dataset:                train_dataset_0.70
	- num of samples:            19
	- batch size:                19
	- unused samples:            0.0
	  └>n_samples-update_per_epochs*batch_size
val dataset:                  validation_dataset_0.20
val {batch size, samples}:    {5, 5}
test dataset:                 test_dataset_0.10
test {batch size, samples}:   {3, 3}
minimizers:                   {'error': {'A': 'out',
                                         'B': 'SamplePart7',
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'params': 'PFir2p'}]
================= nnodely Training =================
|  Epoch   | 

In [6]:
training_parameters = model.trainModel(models='model', train_dataset='dataset', num_of_epochs=10, lr=0.01, shuffle_data=True, train_batch_size=4)

======================== nnodely Model Train Parameters ========================
models:                       'model'
num of epochs:                10
update per epochs:            6.75
└>(n_samples-batch_size)/batch_size+1
shuffle data:                 True
train dataset:                dataset
	- num of samples:            27
	- batch size:                4
	- unused samples:            0.0
	  └>n_samples-update_per_epochs*batch_size
minimizers:                   {'error': {'A': 'out',
                                         'B': 'SamplePart7',
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.01}
optimizer params:             [{'params': 'PFir2p'}]
================= nnodely Training =================
|  Epoch   |       error       |       Total       |
|          |        Loss       |        Loss       |
|          |       train       |       train       |
|--------------------------------------------

To automatically splits the dataset between training set and validation set, just provide the datasets to use inside the `dataset` parameter.

In this way, the parameter `splits` will be used to splits the data accordingly.

In [8]:
training_parameters = model.trainModel(models='model', dataset='dataset', splits=[70, 20, 10], num_of_epochs=10, shuffle_data=True, train_batch_size=4)

TypeError: Modely.trainModel() got an unexpected keyword argument 'dataset'

## Recurrent Training

The recurrent training can take place only when there are recurrent variables, closed-loops or connections between inputs.

The recurrent train can also be performed on the run by setting the `closed_loop` dictionary for closed connections and/or by setting the `connect` dictionary for direct connections.

In case of a recurrent training, the number of prediction horizon window (`prediction_samples`) must be specified. This is used to select for how many steps doing the recurrent loop

The `step` is used to decide whether to skip samples at each epoch. this will ensure a faster training time expecially when the prediction horizon is too high.

In [6]:
target = Input('target')
x = Input('x')
relation = Fir(x.last())
relation.closedLoop(x)
output = Output('out', relation)

test = Modely(visualizer=TextVisualizer(), seed=42)
test.addModel('model', output)
test.addMinimize('error', target.next(), relation)
test.neuralizeModel(0.01)

dataset = {'x': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 'target': [21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]}
test.loadData(name='dataset', source=dataset)

[check_names] The name 'target' is already in defined as NeuObj but it is overwritten.
[check_names] The name 'out' is already in defined as NeuObj but it is overwritten.
================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.01,
          'nnodely_version': '1.5.0',
          'ns': [1, 1],
          'ntot': 2,
          'num_parameters': 1},
 'Inputs': {'target': {'dim': 1, 'ns': [0, 1], 'ntot': 1, 'sw': [0, 1]},
            'x': {'closedLoop': 'Fir7',
                  'dim': 1,
                  'local': 1,
                  'ns': [1, 0],
                  'ntot': 1,
                  'sw': [-1, 0]}},
 'Minimizers': {'error': {'A': 'SamplePart9', 'B': 'Fir7', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Fir7'},
 'Parameters': {'PFir8W': {'dim': 1,
                           'sw': 1,
                           'values': [[0.8822692632675171]]}},
 'Relations': {'Fir7': ['Fir', ['Sample

In [7]:
training_parameters = test.trainModel(train_dataset='dataset', lr=0.01, num_of_epochs=10, train_batch_size=4, prediction_samples=2, step=1, shuffle_data=True)

======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                10
update per epochs:            3
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           2
step:                         1
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            19
	- batch size:                4
	- unused samples:            5
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'error': {'A': 'SamplePart9',
                                         'B': 'Fir7',
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.01}
optimizer params:             [{'params': 'PFir8W'}]
================= nnodely Training =================
|  Epoch   |      

## Set different weight for the minimization functions

use the `minimize_gain` attribute to modify the importance of certain minimization functions by passing a dictionary with the gain factor for each minimizers.

In [8]:
training_parameters = test.trainModel(train_dataset='dataset', minimize_gain={'error':0})

======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                100
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            19
	- batch size:                19
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'error': {'A': 'SamplePart9',
                                         'B': 'Fir7',
                                         'gain': 0,
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'params': 'PFir8W'}]
================

## Setting the Optimizer

There are various ways to configure the optimizer to use during the training process.

This can be achieved by selecting the name of the optimizer to use from the pytorch list and then passing a dictionary of options to configure the desired optimizer

In [9]:
optimizer_defaults = {
            'lr': 0.1,
            'betas': (0.5, 0.99)
        }

training_parameters = test.trainModel(train_dataset='dataset', optimizer='Adam', optimizer_defaults=optimizer_defaults, num_of_epochs=10)

======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                10
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            19
	- batch size:                19
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'error': {'A': 'SamplePart9',
                                         'B': 'Fir7',
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'betas': (0.5, 0.99), 'lr': 0.1}
optimizer params:             [{'params': 'PFir8W'}]
================= nnodely Training ==============

## Early stopping and Best Model

Use one of the built-in early stopping and selection model functions or use a custom one. 

In [10]:
from nnodely.support.earlystopping import early_stop_patience, select_best_model
training_parameters = test.trainModel(train_dataset='dataset', minimize_gain={'error':0}, early_stopping=early_stop_patience, select_model=select_best_model)

======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                100
update per epochs:            1
└>(n_samples-batch_size-prediction_samples+1)/(batch_size+step-1)+1
shuffle _data:                True
early stopping:               early_stop_patience
early stopping params:        {}
prediction samples:           0
step:                         0
closed loop:                  {}
connect:                      {}
train dataset:                dataset
	- num of samples:            19
	- batch size:                19
	- unused samples:            0
	  └>n_samples-prediction_samples-update_per_epochs*(batch_size+step-1)
minimizers:                   {'error': {'A': 'SamplePart9',
                                         'B': 'Fir7',
                                         'gain': 0,
                                         'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           

|  40/100  |       0.e+00      |       0.e+00      |
|  50/100  |       0.e+00      |       0.e+00      |
============================ nnodely Training Time =============================
Total time of Training:       0.07100605964660645


## Train and Analyze

the function `trainAndAnalyze` is a convenient way to do the training and validation at the same time. 

With the following function the user can use a test dataset to assess the quality of the trained model.

In [11]:
test_folder = 'data'
data_struct = ['in1', '', 'target']
model.loadData(name='dataset_test', source=test_folder, format=data_struct, skiplines=1)
model.loadData(name='dataset_val', source=test_folder, format=data_struct, skiplines=1)

training_parameters = model.trainAndAnalyze(models='model', train_dataset='dataset', validation_dataset='dataset_val', train_batch_size=4, test_dataset='dataset_test', test_batch_size=1, num_of_epochs=10, lr=0.01, shuffle_data=True)

============================ nnodely Model Dataset =============================
Dataset Name:                 dataset_test
Number of files:              1
Total number of samples:      27
Shape of in1:                 (27, 5, 1)
Shape of target:              (27, 1, 1)
============================ nnodely Model Dataset =============================
Dataset Name:                 dataset_val
Number of files:              1
Total number of samples:      27
Shape of in1:                 (27, 5, 1)
Shape of target:              (27, 1, 1)
======================== nnodely Model Train Parameters ========================
models:                       ['model']
num of epochs:                10
update per epochs:            6
└>(n_samples-batch_size)/batch_size+1
shuffle _data:                True
train dataset:                dataset
	- num of samples:            27
	- batch size:                4
	- unused samples:            3
	  └>n_samples-update_per_epochs*batch_size
val dataset:         

In [12]:
import pprint
print("Training completed with parameters:", pprint.pprint(training_parameters))

{'add_optimizer_defaults': {},
 'add_optimizer_params': [],
 'all_models': ['model'],
 'closed_loop': {},
 'connect': {},
 'dataset': None,
 'early_stopping': None,
 'early_stopping_params': {},
 'minimize_gain': {},
 'minimizers': {'error': {'A': 'Fir2', 'B': 'SamplePart4', 'loss': 'mse'}},
 'models': ['model'],
 'n_samples_test': 27,
 'n_samples_train': 27,
 'n_samples_val': 27,
 'name': None,
 'num_of_epochs': 10,
 'optimizer': 'Adam',
 'optimizer_defaults': {'lr': 0.01},
 'optimizer_params': [{'params': 'PFir3W'}],
 'prediction_samples': -1,
 'select_model': 'last',
 'select_model_params': {},
 'shuffle_data': True,
 'splits': [100, 0, 0],
 'step': 0,
 'test_batch_size': 1,
 'test_step': 0,
 'test_tag': 'dataset_test',
 'train_batch_size': 4,
 'train_dataset': 'dataset',
 'train_step': 0,
 'train_tag': 'dataset',
 'unused_samples': 3,
 'update_per_epochs': 6,
 'val_batch_size': 27,
 'val_step': 0,
 'val_tag': 'dataset_val',
 'validation_dataset': 'dataset_val'}
Training completed w